## 한국어 영화 리뷰 분석

200개 영화 리뷰 100개씩을 가져온 데이터

1~10점까지의 평점

중립(5-8)은 제외, (1-4)는 긍정, (9-10)은 부정

In [0]:
!git clone https://github.com/e9t/nsmc.git
  ##파일에 생긴다! 깃 꺼를 업로드할 때 이런식으로~

In [0]:
!cat ./nsmc/ratings_train.txt | head -n 10
##id 는 sentence에 대한 고유 id
## 라벨은 0이면 부정, 1이면 긍정 ... 조금의 오류는 존재...

In [0]:
with open("./nsmc/ratings_train.txt", "r") as f:
  file = []
  for s in f:
    file.append(s.strip().split('\t'))

In [0]:
file[:10]

In [0]:
import pandas as pd

In [0]:
train = pd.read_csv("./nsmc/ratings_train.txt",sep='\t')

In [0]:
train.head()

In [0]:
"""
def read_data(filename):
  ##파일 오픈
  ##내가 원하는 리스트 형식 처리해주는 code
  with open(filename,'r') as f:
    data = []
    for line in f:
      data.append(line.strip().split('\t'))
    data = data[1:]
  return data
  """
##구데기.... 다른 함수로 바꾸자!

In [0]:
with open("./nsmc/ratings_train.txt", 'r') as f:
  data = [line.split('\t') for line in f.read().splitlines()]

In [0]:
data[:3]

In [0]:
def read_data(filename):
  with open(filename,'r') as f:
    data = [line.split('\t') for line in f.read().splitlines()]

    data = data[1:]
  return data

In [0]:
train_data = read_data("./nsmc/ratings_train.txt")
test_data = read_data("./nsmc/ratings_test.txt")

In [0]:
train_data[:10]

In [0]:
len(train_data), len(train_data[0]),len(test_data), len(test_data[0]),

In [0]:
!pip install konlpy

In [0]:
import konlpy
konlpy.__version__

In [0]:
from konlpy.tag import Okt

okt = Okt()

In [0]:
train_data = train_data[:10000]
test_data = test_data[:10000]

In [0]:
okt.pos(train_data[0][1], norm = True, stem=True)

In [0]:
"""
def tan_a(data):
  tag=[]
  for i in range(0,len(data)):
    tag.append(okt.pos(data[i][1], norm=True, stem=True))
    
  return tag
  """
##이 또한 구데기...

In [0]:
def tokenize(doc):
  ##norm은 정규화, stem은 근어로 표시하기를 나타냄
  return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]
## / 는 품사랑 구분 짓기 위해서 리스트로 되어 있으니까 조인을 시키겠다~

In [52]:
train_data[0]

['9976970', '아 더빙.. 진짜 짜증나네요 목소리', '0']

In [53]:
tokenize(train_data[0][1])

['아/Exclamation',
 '더빙/Noun',
 '../Punctuation',
 '진짜/Noun',
 '짜증나다/Adjective',
 '목소리/Noun']

In [0]:
def make_token(doc):
  return ['/'.join(t) 
          for data in doc
          for t in okt.pos(data[1], norm=True, stem=True)
          ]
# 함수만들기 -okt 이용해서 형태소분석 할 수 있도록 okt.pos 사용
# 이 함수를 이용해서 tokenize가 되도록 작성!

In [0]:
train_pos = tokenize(train_data)
test_pos = tokenize(test_data)

In [0]:
import json
import os
from pprint import pprint

def tokenize(doc):
  ##norm은 정규화, stem은 근어로 표시하기를 나타냄
  return ['/'.join(t) for t in okt.pos(doc, norm=True, stem=True)]

# 함수만들기 -okt 이용해서 형태소분석 할 수 있도록 okt.pos 사용
# 이 함수를 이용해서 tokenize가 되도록 작성!

if os.path.isfile("./nsmc/train_doc.json"):
  with open("./nsmc/train_doc.json") as f:
    train_docs = json.load(f)
    
  with open("./nsmc.test_doc.json") as f:
    test_docs = json.load(f)
    
else : 
  #train_data에 있는걸 한줄 씩 받아서 리스트로 만들기
  train_docs = [(tokenize(row[1]), row[2]) for row in train_data]
  test_docs = [(tokenize(row[1]), row[2]) for row in test_data]
  
  with open("./nsmc/train_doc.json",'w', encoding='utf-8') as make_file:
    json.dump(train_docs, make_file, ensure_ascii=False, indent='\t')
  
  with open("./nsmc/test_doc.json",'w', encoding='utf-8') as make_file:
    json.dump(test_docs, make_file, ensure_ascii=False, indent='\t')
    
##if 파일이 있다면, 읽은 다음에 json파일로 생성
##else train, test파일을 tokenize함수를 적용하고, json파일로 저장

In [0]:
pprint(train_docs[:2])

In [0]:
tokens = [t for d in train_docs for t in d[0]]
print(len(tokens))

In [0]:
import nltk
text= nltk.Text(tokens, name="NMSC")
print(text)
##id 값을 주기 위해 nltk를 이용!

In [0]:
print(len(text.tokens))
##결국 15만개!

In [0]:
print(len(set(text.tokens)))

In [0]:
pprint(text.vocab().most_common(10))

In [0]:
##10000개 만을 가지고 사전만들기!

# 상위 10000개 token을 벡터화시킴
selected_words = [f[0] for f in text.vocab().most_common(10000)]

# Bag of words를 이용하여 count vector를 만드는 함수
# 결국은 빈도세기 위해서!!
def term_frequency(doc):
  return [doc.count(word) for word in selected_words]


# x에는 count vecto가 적용된 문장을
# y에서는 정답 셋을
train_x = [term_frequency(d) for d, _ in train_docs]
test_x = [term_frequency(d) for d, _ in test_docs]
train_y = [c for _, c in train_docs]
test_y = [c for _, c in test_docs]

# train의 라벨은 숨긴다!! 왜냐하면 비교하기 위해

In [0]:
selected_words[:100]

In [0]:
import numpy as np
x_train = np.asarray(train_x).astype('float32')
x_test = np.asarray(test_x).astype('float32')
y_train = np.asarray(train_y).astype('float32')
y_test = np.asarray(test_y).astype('float32')
##이렇게 데이터를 float로 바꿔주면 데이터 전처리는 끝!!

Unit : 신경망 데이터를 얼마나 복잡하게 할 것인가?

몇 개의 layer를 사용할 것인가??

pdf확인하기!

In [0]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow.keras import losses
from tensorflow.keras import metrics
model = models.Sequential()
model.add(layers.Dense(64, activation='relu',
input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

In [0]:
# 손실 함수로는 binary_crossentropy를 사용
# RMSProp 옵티마이저를 통해서 gradient desent을 진행
# 또한 배치 사이즈를 512로, 에포크를 10번으로 학습
model.compile(optimizer=optimizers.RMSprop(lr=0.001),
loss=losses.binary_crossentropy,
metrics=[metrics.binary_accuracy])
model.fit(x_train, y_train, epochs=10, batch_size=512)
results = model.evaluate(x_test, y_test)

In [0]:
results

In [0]:
def predict_pos_neg(review):
  token = tokenize(review)
  tf = term_frequency(token)
# 데이터 형태 맞추기 위해 np.expand_dims 메서드를 이용해 array의 축을 확장
  data = np.expand_dims(np.asarray(tf).astype('float32'), axis=0)
  score = float(model.predict(data))
# 최종 확률이 0.5 보다 크면 긍정이고, 그렇지 않으면 부정이라고 예측
  if(score > 0.5):
    print("[{}]는 {:.2f}% 확률로 긍정 리뷰이지 않을까추측해봅니다.^^\n"\
          .format(review, score * 100))
  else:
    print("[{}]는 {:.2f}% 확률로 부정 리뷰이지 않을까추측해봅니다.^^;\n"\
          .format(review, (1 - score) * 100))

In [0]:
predict_pos_neg("올해 최고의 영화! 세 번 넘게 봐도 질리지가않네요.")
predict_pos_neg("배경 음악이 영화의 분위기랑 너무 안맞았습니다. 몰입에 방해가 됩니다.")
predict_pos_neg("주연 배우가 신인인데 연기를 진짜 잘 하네요.몰입감 ㅎㄷㄷ")
predict_pos_neg("믿고 보는 감독이지만 이번에는 아니네요")
predict_pos_neg("주연배우 때문에 봤어요")

여기서는 결국 bag of words가 필요했다

내가 필요한 json 파일이 없다면 만들어서 생성도 해야하고 있다면 패스도 해야한다.

그리고 문제에 맞게 설계를 해야할 것이다.

1. git clone을 통해 데이터를 가져와서 확인을 함
2. 파일을 로드해서 데이터를 생성 -> 데이터를 잘 생성했는지 check
3. konlpy의 okt를 통해 생성 ...속도가 좋고, 성능도 어느정도 나오고, 신조어 처리가 가능
4. tokenize를 실시함! ... 형태소 분석을 통해서 이 문장이 어떤 단어, 어떻게 라벨링이 되어있는지 check하고 json파일 생성! ... if else
5. 토크나이즈 bag of words를 위해 전체 토큰을 생성함! ... 하나의 리스트로 생성 ... 리스트로 만든 이유는? 15만개를 봄! 
6. 그리고 10000개만 뽑아서 senetence에서 word와 매칭되어 있으면 카운트를 실시!
7. float로 텐서플로우에 넣기 위해 다시 생성
8. layer를 3개 생성 2개는 Lelu, 1개는 Sigmoid ... 01로 하기 위해!
9. 그리고 결과를 본다.
10. 함수를 이용하여, 긍정.부정 리뷰를 분리함